In [ ]:
from serial.tools.list_ports import comports
for port in comports():
    print(port)

In [2]:
import subprocess

def check_git_status():
    try:
        # Check if the local branch is up to date with the remote branch
        result = subprocess.run(['git', 'remote', 'update'], capture_output=True, text=True)
        result = subprocess.run(['git', 'status', '-uno'], capture_output=True, text=True)
        if "Your branch is up to date" in result.stdout:
            print("Code is up to date with the remote repository.")
        elif "Your branch is behind" in result.stdout:
            print("!!!!!!!!!!!!! There are changes in the remote repository that are not in the local repository. !!!!!!!!!!!!!")
        elif "Your branch is ahead" in result.stdout:
            print("!!!!!!!!!!!!! The local repository is ahead of the remote repository. !!!!!!!!!!!!!")
        else:
            print("The local repository has diverged from the remote repository.")
    except Exception as e:
        print(f"An error occurred while checking the git status: {e}")

# Check the git status
check_git_status()


The local repository is up to date with the remote repository.


In [ ]:
import os
import sys
import time
import csv
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from Keithley617.Keithley617 import Keithley617 as smu

# Set parameters
start_voltage = -10 #V
end_voltage = 100 # V
step_size = 1 # V/step
scan_rate = 1 # V/s
sample_number = 5 #  number of Samples for standard deviation calculation
max_samples = 30 # Timeout for resampling
standard_deviation_threshold = 0.1 #required standard deviation of measurement to proceed.

if start_voltage > end_voltage:
    step_size = -step_size
source_voltages = np.arange(start_voltage, end_voltage+step_size, step_size) # prepare voltage sources

total_time = abs(end_voltage - start_voltage) / scan_rate
delay_time =  total_time / len(source_voltages)

# Set the filename
file_name = input('Enter file name: ')
filename = os.path.join(file_name + '.csv')

# Check Parameters
print(f'File name: {file_name}.csv')
print(f'Range: from {start_voltage} V to {end_voltage} V')
print(f'Scan rate: {scan_rate} V/s')


# Open a connection to the Keithley 617
smu = smu('/dev/cu.usbserial-PX4IFKTF')

# Make sure if you start or not
START = input('Press Enter to Start')
if START == '':
    print("\n Let's get started :)")
    pass
else:
    sys.exit(0)

# Initialize the time and current arrays
times = []
currents = []
voltages = []

# Set up the real-time plot
plt.ion()
fig = plt.figure(figsize=(12,8))
plt.rcParams["font.size"] = 20
plt.xlabel('Voltage (V)')
plt.ylabel('Current (A)')

if smu:
    # Set the parameters source to properly measure current
    smu.reading_mode('electrometer')
    smu.zero_check('off')
    smu.zero_correct('enabled')
    smu.set_function('amps')
    smu.data_format('without_prefix')
    smu.set_range('R0') # R0 - auto
    
    # Start the measurement and real-time plot
    start_time = time.perf_counter()
    
    # Turn on source output
    smu.source_output('on')
    for voltage in source_voltages:
        round_start = time.perf_counter()
        # Set the voltage source
        smu.set_voltage(voltage)
        clear_output(wait=True)
        samples = []
        #Take sample_number initial samples
        for _ in range(sample_number):
            # Measure current with retry mechanism
            current = None
            for _ in range(3):  # retry up to 3 times
                try:
                    current = smu.measure_current()
                    if current is not None:
                        break
                except Exception as e:
                    print(f"Error measuring current: {e}")
                    time.sleep(0.1)  # brief delay before retrying

            if current is None:
                print("Failed to measure current after 3 attempts")
                continue
            samples.append(current)
        
        # Measure the standard deviation of the last sample_number samples and continue if it is below the standard deviation threshold. Otherwise, keep sampling.
        # Print an error if we exceed the max number of measurements (too unstable to achieve standard deviation threshold value)
        measurementcount = sample_number
        while True:
            standard_deviation = np.std(samples[-sample_number])
            if standard_deviation<standard_deviation_threshold:
                break
            # Measure current with retry mechanism
            current = None
            for _ in range(3):  # retry up to 3 times
                try:
                    current = smu.measure_current()
                    if current is not None:
                        break
                except Exception as e:
                    print(f"Error measuring current: {e}")
                    time.sleep(0.1)  # brief delay before retrying

            if current is None:
                print("Failed to measure current after 3 attempts")
                continue
            samples.append(current)
            measurementcount += 1
            if measurementcount > max_samples:
                print (f"!!ERROR!! Measurement failed to achieve desired standard deviation of {standard_devation_threshold} with applied voltage {voltage} in {max_samples} samples")
                break

        times.append((time.perf_counter() - start_time))
        currents.append(current)
        voltages.append(voltage)
        print(f'Time: {times[-1]:.2f} s')
        print(f'Voltage: {voltage:.4g} V')
        print(f'Current: {current:.4g} A')
        plt.plot(voltages[-100:], currents[-100:], linestyle='-', marker='o', label='Current', color='blue')
        plt.xlabel('Voltage (V)', fontsize=18)
        plt.ylabel('Current (A)', fontsize=18)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.grid(True)
        display(fig)
        if (delay_time - (time.perf_counter() - round_start)) > 0:
            time.sleep(delay_time - (time.perf_counter() - round_start))
        else:
            pass              
    clear_output(wait=True)
    
    # Turn off source output
    smu.source_output('off')
    
    # Close connection to Keithley 617
    smu.disconnect

# Create a CSV file for saving the data
with open(filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Time (s)', 'Current (A)', 'Voltage (V)'])
    for i in range(len(times)):
        csvwriter.writerow([times[i], currents[i], voltages[i]])

# Check that the CSV file was created successfully
try:
    with open(filename, 'r') as csvfile:
        pass
except:
    print("Error: Could not create CSV file")

print("Program completed")